# 2D Nuclear Segmentation with Feature Pyramid Networks

In [1]:
import os
import errno

import numpy as np

import deepcell

In [2]:
# Download the data (saves to ~/.keras/datasets)
filename = 'HeLa_S3.npz'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.hela_s3.load_data(filename)

print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

X.shape: (5760, 216, 256, 1)
y.shape: (5760, 216, 256, 1)


### Set up filepath constants

In [3]:
# the path to the data file is currently required for `train_model_()` functions

# NOTE: Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))

DATA_FILE = os.path.join(DATA_DIR, filename)

# confirm the data file is available
assert os.path.isfile(DATA_FILE)

In [4]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

### Set up training parameters

In [5]:
from tensorflow.keras.optimizers import SGD, Adam
from deepcell.utils.train_utils import rate_scheduler

model_name = 'fpn_model'
backbone = 'resnet50'  # vgg16, vgg19, resnet50, densenet121, densenet169, densenet201

n_epoch = 1
batch_size = 1
test_size = 0.1
norm_method = "std"

# Optimizer settings
lr = 1e-2
optimizer = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=lr, decay=0.99)

# Augmentation settings
rotation_range = 180
flip = True
shear = False
zoom_range = (0.5, 2.0)
test_zoom = [0.5, 0.8, 1.0, 1.2, 2.0]
nsteps = 500

# n_conv_filters = 32
# n_dense_filters = 128

transform = "watershed"
distance_bins = 4
n_features = distance_bins
erosion_width = 0

### Create model

In [6]:
from deepcell.model_zoo import FPNet

model = FPNet(backbone=backbone,
             input_shape=X_train.shape[1:])

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:264: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


(?, 4, 4, 256)
(?, 7, 8, 256)
(?, 14, 16, 256)
(?, 27, 32, 256)
(?, 32, 32, 128)
(?, ?, ?, 128)
(?, ?, ?, 128)
(?, ?, ?, 128)


In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 216, 256, 1)  0                                            
__________________________________________________________________________________________________
image_normalization2d (ImageNor (None, 216, 256, 1)  3721        input_1[0][0]                    
__________________________________________________________________________________________________
tensor_product (TensorProduct)  (None, 216, 256, 3)  6           image_normalization2d[0][0]      
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 222, 262, 3)  0           tensor_product[0][0]             
__________________________________________________________________________________________________
conv1 (Con